># Day 3 - Binary diagnostic
>
>The submarine has been making some odd creaking noises, so you ask it to produce a diagnostic report just in case.
>
>The diagnostic report (your puzzle input) consists of a list of binary numbers which, when decoded properly, can tell you many useful things about the conditions of the submarine. The first parameter to check is the power consumption.
>
>You need to use the binary numbers in the diagnostic report to generate two new binary numbers (called the gamma rate and the epsilon rate). The power consumption can then be found by multiplying the gamma rate by the epsilon rate.
>
>Each bit in the gamma rate can be determined by finding the most common bit in the corresponding position of all numbers in the diagnostic report. For example, given the following diagnostic report:
>
>```
>00100
>11110
>10110
>10111
>10101
>01111
>00111
>11100
>10000
>11001
>00010
>01010
>```
>
>Considering only the first bit of each number, there are five 0 bits and seven 1 bits. Since the most common bit is 1, the first bit of the gamma rate is 1.
>
>The most common second bit of the numbers in the diagnostic report is 0, so the second bit of the gamma rate is 0.
>
>The most common value of the third, fourth, and fifth bits are 1, 1, and 0, respectively, and so the final three bits of the gamma rate are 110.
>
>So, the gamma rate is the binary number 10110, or 22 in decimal.
>
>The epsilon rate is calculated in a similar way; rather than use the most common bit, the least common bit from each position is used. So, the epsilon rate is 01001, or 9 in decimal. Multiplying the gamma rate (22) by the epsilon rate (9) produces the power consumption, 198.
>
>Use the binary numbers in your diagnostic report to calculate the gamma rate and epsilon rate, then multiply them together. What is the power consumption of the submarine? (Be sure to represent your answer in decimal, not binary.)

In [125]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

df = pd.read_csv('input_day3.txt').astype("string") # No numbers please, strings
df.head()

,Values
0,101111101000
1,110110011100
2,10111111000
3,10101000001
4,11000000001


In [126]:
# Turns out the MSB's zeroes are not present, first correct that
values = df['Values'].tolist()

# Find the longest string with a comprehension
longest = max((len(x) for x in values))

# Add the required number of leading zeroes
for i in range(len(values)):
    length = len(values[i])
    values[i] = (longest-length) * '0' + values[i]

# Control that everything went correctly
outliers = [x for x in values if len(x) != longest]
assert len(outliers) == 0, f"Oh no, some elements have not a size of {{longest}}"

In [137]:
# We can now get started with the job
# Transpose a list of string
def transpose(array):
    res = []    
    # Columns
    for column in range(len(array[0])):
        # Lines
        tmp = []            
        for ligne in range(len(array)):
            tmp.append(array[ligne][column])

        res.append(tmp)
    return res

#transposed = transpose(['111', '100', '101', '001'])
transposed = transpose(values)

In [155]:
# Convert a list to a string
def list2string(list):
    return ''.join([elem for elem in list])

# Get the prevalent one or zero from a list
def mostPresent(x):    
    ones = len([elem for elem in x if elem == '1'])
    zeroes = len([elem for elem in x if elem == '0'])
    assert (zeroes + ones) == len(x)
    return '1' if ones > zeroes else '0'

transposedAsString = [list2string(elem) for elem in transposed]
mostPresentBits = [mostPresent(elem) for elem in transposedAsString]

# Convert to a number epsilon
epsilon = int(''.join([x for x in mostPresentBits]), 2)

# Compute gamma as every bit inverted
tmp = []
for x in mostPresentBits:   
    tmp.append('1' if x == '0' else '0')    

# Convert to number
gamma = int(''.join([x for x in tmp]), 2)

print(f"Gamma = {gamma}, epsilon = {epsilon}")
print(f"Product is {gamma * epsilon}")

Gamma = 3811, epsilon = 284
Product is 1082324
